# AddOn calculation

Most of the SA-CCR logic is hidden inside the AddOn calculation. At first it is important to define the following four data parameters:

##### $M_i$
> Maturity of the derivative contract. If the underlying of a derivative is another derivative - e.g. in the case of a swaption the maturity date of the underlying needs to be chosen.

##### $S_i$

> For interest rate and credit derivatives the start date of the time periodreferenced by an interst rate or credit contract. If the derivtives underlying is another interest rate or credit intsrument (eg swaption or bond option) $S_i$ is the start date of the underlying instead.

##### $E_i$

> Defined as $S_i$ but referencing the end date instead of the start date.

##### $T_i$

> For options across all asset classes this is the latest contractual exercise date.

In [1]:
from math import log, sqrt
import numpy
from pandas import DataFrame
from scipy import optimize
from scipy.stats import norm
from collateralAgreement import CollateralAgreement, Margining, Clearing, Tradecount, Dispute
from instruments.equity_instruments.equityOption import EquityOption
from IPython.display import display, Markdown, Latex
from instruments.interestRateInstrument.irs import IRS
from instruments.interestRateInstrument.ois import OIS
from instruments.interestRateInstrument.swaption import Swaption
from jupyterUtils import export
import QuantLib as ql
from marketdata import init_marketdata
from marketdata.EquitySpot import EquitySpot
from marketdata.fx_spot import FxSpot
from marketdata.interestRateIndices import InterestRateIndex
from sa_ccr.sa_ccr import SA_CCR
from utilities.Enums import TradeType, TradeDirection, Stock, SwapDirection
asdf =1



## Trade level adjusted notional

Each trade $i$ has a trade level adjusted notional $d_i^a$ assigned to it. This is calculated differently for the different asset classes.

#### Interest rate and credit derivatives

The notional of the trade is usually a well defined value in domestic currency for interest rate and credit derivatives. It is multiplied by a supervisory duration factor. The basic idea is, that the value of the derivative can change more the longer the remaining

\begin{align*}
d_i &= \text{Notional}_i * SD_i \\
\\
\text{where} \qquad SD_i &=\frac{\exp\left(-0.05 * S_i\right)-\exp\left(-0.05 * E_i\right)}{0.05}
\end{align*}

#### FX derivatives

While the wording in the BCBS paper is a bit more specific we will just assume that every FX traded derivative has a USD leg and set the notional equal the to USD notional.

#### Equity and commodity derivatives

The notional is defined as the price of the underlying. Therefore, it fluctuates over time.

#### Notional of exotic derivatives

For more exotic derivatives which do have adjustable notionals, resetting notionals etc. detailed handling of the notional is defined in paragraph 158.


Within this thesis we investigate only equity and interest rate derivatives. For these we can make a few exemplary calculations of the trade level adjusted notional.

For equity trades determining the trade level adjusted notional is trivial as it always is the spot price of the underlying. As an example consider the two trades defined below:

In [2]:
#When the strike is not set explicitly an at the money option is created with K = S(t0)
eqOption1 = EquityOption(maturity = ql.Period(1, ql.Years),
                         tradeType= TradeType.CALL,
                         tradeDirection= TradeDirection.LONG,
                         underlying= Stock.ADS)

eqOption2 = EquityOption(maturity = ql.Period(1,   ql.Years),
                         tradeType= TradeType.PUT,
                         tradeDirection= TradeDirection.SHORT,
                         underlying= Stock.ADS,
                         strike = 60)

In [3]:
display(Markdown('Let the spot price of Adidas stock be %d.' % EquitySpot.ADS.value.value() + ' Then, the adjusted notional of ``eqOption1``, an at the money call on Adidas, is %d' % SA_CCR.trade_level_adjusted_notional(eqOption1) + ' and the adjusted notional of ``eqOption2``, a short in the money put on Adidas, is also %d.' % SA_CCR.trade_level_adjusted_notional(eqOption2)))

Let the spot price of Adidas stock be 42. Then, the adjusted notional of ``eqOption1``, an at the money call on Adidas, is 42 and the adjusted notional of ``eqOption2``, a short in the money put on Adidas, is also 42.

For interest rate derivatives such as interest rate swaps or swaptions on the other hand, the notional is adjusted by the supervisory duration factor. As the supervisory duration depends on *S* and *E* it is important to understand how these are determined for the different interest rate derivatives.

| Trade  Type              | *S*                               | *E*                                   |
|--------------------------|-----------------------------------|---------------------------------------|
| **Interest Rate Swap**   | Current date                      | Maturity date                         |
| **Forward starting IRS** | Start date of the underlying swap | Maturity date  of the underlying swap |
| **Swaption**             | Start date of the underlying swap | Maturity data of the underlying swap  |

Let's check if this relationship holds for the implementation

In [4]:
testIRS = IRS(notional=100,
               timeToSwapStart=ql.Period(2, ql.Days),
               timeToSwapEnd=ql.Period(3, ql.Years),
               swapDirection= SwapDirection.PAYER,
               index = InterestRateIndex.EURIBOR6M
               )

testOIS = OIS(notional=100,
              timeToSwapStart=ql.Period(2, ql.Days),
              timeToSwapEnd=ql.Period(3, ql.Years),
              swapDirection= SwapDirection.RECEIVER,
              index = InterestRateIndex.FEDFUNDS
              )

testFWSIRS = IRS(notional=100,
               timeToSwapStart=ql.Period(1, ql.Years),
               timeToSwapEnd=ql.Period(3, ql.Years),
               swapDirection= SwapDirection.PAYER,
               index = InterestRateIndex.EURIBOR6M
               )

testFWSOIS = OIS(notional=100,
              timeToSwapStart=ql.Period(1, ql.Years),
              timeToSwapEnd=ql.Period(3, ql.Years),
              swapDirection= SwapDirection.RECEIVER,
              index = InterestRateIndex.FEDFUNDS
              )

testSwaption = Swaption(underlyingSwap=testFWSIRS,
                        optionMaturity=ql.Period(1, ql.Years),
                        tradeDirection=TradeDirection.LONG)

testIRS2 = IRS(notional=100,
               timeToSwapStart=ql.Period(2, ql.Days),
               timeToSwapEnd=ql.Period(2, ql.Years),
               swapDirection=SwapDirection.PAYER,
               index = InterestRateIndex.EURIBOR6M)

print(SA_CCR.trade_level_adjusted_notional(testIRS))
print(SA_CCR.trade_level_adjusted_notional(testOIS))
print(SA_CCR.trade_level_adjusted_notional(testFWSIRS))
print(SA_CCR.trade_level_adjusted_notional(testFWSOIS))
print(SA_CCR.trade_level_adjusted_notional(testSwaption))
print(SA_CCR.trade_level_adjusted_notional(testIRS2))

278.02856874767866
278.02856874767866
181.04289615131242
181.04289615131242
181.04289615131242
189.76968552587525


## Supervisory delta adjustments: $\delta_i$

For linear derivatives $\delta$ is 1 for long derivatives and -1 for short derivatives.

For options $\delta$ is defined as under Black-Scholes:

\begin{align*}
\delta_{\text{long Call}} &= +\Phi\left(\frac{\ln\left(P_i / K_i \right) + 0.5 * \sigma_i^2 * T_i}{\sigma_i * \sqrt{T_i}}\right) \\
\\
\text{where} \qquad \Phi &: \text{standard normal cdf} \\
\sigma_i &: \text{supervisory volatility as defined in Table 2 in paragraph 183}
\end{align*}

This delta is multiplied by -1 in case of a long Put option or a short Call option. This formula is used for both, equity options and swaptions.

No detail is given at this point on the delta calculation of CDO tranches as these are not in the scope of this thesis.

In the case of an european equity option the parametrization is quite straight forward.

$\sigma_i$: 1.2 is the supervisory volatility for a single stock option

$K_i$: The strike of the option

$P_i$: The spot price of the underlying stock

$T_i$: The maturity of the option

A swaption on the other hand is parametrized as follows for calculation of its supervisory delta:

$\sigma_i$: 0.5 is the supervisory volatility for any interst rate option.

$K_i$: The strike of the option is the fixed rate of the underlying swap

$P_i$: Is the current par rate of the underlying (forward starting) swap

$T_i$: The maturity of the option. Please note the difference to $E_i$ used for calculation of the adjusted notional, which is the maturity of the underlying swap.


SA-CCR uses the same Black-Scholes based formula for Swaps as it uses for Equities. It differentiates options in two dimensions. Whether they are *bought* or *sold* and whether they are *Call* or *Put* options (Compare paragraph 159).

SA-CCR defines an option as a call option if it rises in value as the underlying rises in value. A fixed payer swap rises in value as the underlying interest rate rises in value. Therefore, an option to buy a fixed payer swap at a predetermined strike also rises in value as the underlying interest rate rises in value. Therefore, a swaption on a payer swap is considered a *Call* under SA-CCR, while a swaption on a receiver swap is considered a *Put*.


Let's check, if we actually calculate delta correctly for our options.

In [5]:
#manual calculation yields
p_eq = EquitySpot.ADS.value.value()
K = eqOption1.K
print(K)
sigma = 1.2
T = eqOption1.t
print(T)

def quick_calc(P, K, sigma, T):
    return norm.cdf((log(P/K)+0.5*sigma**2*T)/(sigma*sqrt(T)))

print(quick_calc(p_eq, K, sigma, T))

42.0
1.0
0.7257468822499265


In [6]:
p_sw = testFWSOIS.get_par_rate()
print(p_sw)
K = testFWSOIS.get_fixed_rate()
print(K)
sigma = 0.5
T = testSwaption.t
print(T)

print(quick_calc(p_sw, K, sigma, T))

0.01901615980132947
0.019016159801329445
1.0
0.5987063256829248


In [7]:
#now for a swaption
print(testSwaption.underlying_swap.ql_instrument.fairRate())
P = testSwaption.underlying_swap.get_par_rate()
print(testSwaption.underlying_swap.get_fixed_rate())
K= testSwaption.underlying_swap.get_fixed_rate()


-0.00113185568476534
-0.0011318556847654294


In [8]:
display(Markdown('For the at the money option ``eqOption1`` that was set up above we yield a supervisory delta adjustment of %.4f.' % SA_CCR.calculate_sa_ccr_delta(eqOption1)))

For the at the money option ``eqOption1`` that was set up above we yield a supervisory delta adjustment of 0.7257.

For an examplary short european swaption that has a par swap as underlying (i.e. the NPV of the swap is 0) that is set up as follows:

In [9]:
swap = IRS(notional=100,
           timeToSwapStart=ql.Period(1, ql.Years),
           timeToSwapEnd=ql.Period(3, ql.Years),
           swapDirection=SwapDirection.PAYER,
           index=InterestRateIndex.EURIBOR6M
          )

swaption = Swaption(underlyingSwap=swap,
                    optionMaturity=ql.Period(1, ql.Years),
                    tradeDirection=TradeDirection.SHORT)

SA_CCR.calculate_sa_ccr_delta(swaption)

-0.5987063256828626

In [10]:
display(Markdown('we yield a regulatory delta of %.4f.' % SA_CCR.calculate_sa_ccr_delta(swaption)))


we yield a regulatory delta of -0.5987.

In [11]:
print(SA_CCR.calculate_sa_ccr_delta(testIRS))
print(SA_CCR.calculate_sa_ccr_delta(eqOption1))
print(SA_CCR.calculate_sa_ccr_delta(testSwaption))

1
0.7257468822499265
0.5987063256828626


Everything checks out. We can go ahead

In [13]:
export("building_SA_CCR_2.ipynb")